In [1]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from datasets import load_dataset

# Load the MMLU dataset
dataset = load_dataset("cais/mmlu","all")

# Explore the dataset
print(dataset)


Generating test split:   0%|          | 0/14042 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1531 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/285 [00:00<?, ? examples/s]

Generating auxiliary_train split:   0%|          | 0/99842 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 14042
    })
    validation: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 1531
    })
    dev: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 285
    })
    auxiliary_train: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 99842
    })
})


In [3]:
import pandas as pd
test = pd.DataFrame(dataset['test'])

In [4]:
test['Name'] = 'MMLU_test'

In [5]:
test.head()

,question,subject,choices,answer,Name
0,Find the degree for the given field extension ...,abstract_algebra,"[0, 4, 2, 6]",1,MMLU_test
1,"Let p = (1, 2, 5, 4)(2, 3) in S_5 . Find the i...",abstract_algebra,"[8, 2, 24, 120]",2,MMLU_test
2,Find all zeros in the indicated finite field o...,abstract_algebra,"[0, 1, 0,1, 0,4]",3,MMLU_test
3,Statement 1 | A factor group of a non-Abelian ...,abstract_algebra,"[True, True, False, False, True, False, False,...",1,MMLU_test
4,Find the product of the given polynomials in t...,abstract_algebra,"[2x^2 + 5, 6x^2 + 4x + 6, 0, x^2 + 1]",1,MMLU_test


In [6]:
for k,v in test.iloc[10].items():
    print("The Column name:" + k)
    print("The Value:" + str(v))

The Column name:question
The Value:Statement 1 | If H is a subgroup of G and a belongs to G then |aH| = |Ha|. Statement 2 | If H is a subgroup of G and a and b belong to G, then aH and Hb are identical or disjoint.
The Column name:subject
The Value:abstract_algebra
The Column name:choices
The Value:['True, True', 'False, False', 'True, False', 'False, True']
The Column name:answer
The Value:2
The Column name:Name
The Value:MMLU_test


In [7]:
# 1: Combine the question and choices into a single column 'Question'
df = test
df['Question'] = df.apply(lambda row: f"{row['question']} The options are: " + 
                          ', '.join([f"{chr(65+i)}) {choice}" for i, choice in enumerate(row['choices'])]), axis=1)

# 2: Rename the columns
df.rename(columns={'subject': 'Category', 'answer': 'Correct Answer'}, inplace=True)

# 3: Change the value of 'Correct Answer' to A, B, C, D
df['Correct Answer'] = df['Correct Answer'].apply(lambda x: chr(65 + x))

# Selecting the required columns
df_final = df[['Name', 'Category', 'Question', 'Correct Answer']]

df_final

,Name,Category,Question,Correct Answer
0,MMLU_test,abstract_algebra,Find the degree for the given field extension ...,B
1,MMLU_test,abstract_algebra,"Let p = (1, 2, 5, 4)(2, 3) in S_5 . Find the i...",C
2,MMLU_test,abstract_algebra,Find all zeros in the indicated finite field o...,D
3,MMLU_test,abstract_algebra,Statement 1 | A factor group of a non-Abelian ...,B
4,MMLU_test,abstract_algebra,Find the product of the given polynomials in t...,B
...,...,...,...,...
14037,MMLU_test,world_religions,What has been a central focus of religious tra...,A
14038,MMLU_test,world_religions,To whom did ordinary folk appeal during a dro...,C
14039,MMLU_test,world_religions,The theological term homoousios means which o...,B
14040,MMLU_test,world_religions,"According to the Japanese origin myth, who giv...",B


In [9]:
df_final.to_csv('../preprocessed/MMLU_test.csv',index = False)